# **Doublet Panel Method**

In [1]:
include("../src/PanelMethods.jl")
include("../src/FoilParametrization.jl")
using Revise
using .PanelMethods: DoubletPanel2D, Uniform2D, doublet_panels, collocation_point, split_panels, grid_data, aero_coefficients, pressure_coefficient, ×
using .FoilParametrization: read_foil, cosine_foil, kulfan_CST, naca4, linspace
using BenchmarkTools
using Plots
plotly()

Plots.PlotlyBackend()

### Kulfan CST Airfoil

In [2]:
alpha_u = [0.2, 0.3, 0.2, 0.15, 0.2]
alpha_l = [-0.2, -0.1, -0.1, -0.001, -0.02]
alphas = [alpha_u alpha_l]
dzs = (1e-4, 1e-4)
airfoil = kulfan_CST(alphas, dzs, 0.1, 100);
# airfoil = naca4((2, 4, 1, 2), 50, sharp_trailing_edge = true);
# airfoil = cosine_foil(read_foil("../airfoil_database/clarky.dat"), n = 40);

399×2 Array{Float64,2}:
 1.0       0.0001
 0.999938  9.99977e-5
 0.999751  0.000100037
 0.999439  0.000100258
 0.999003  0.000100891
 0.998443  0.000102257
 0.997759  0.000104766
 0.99695   0.000108912
 0.996018  0.000115272
 0.994962  0.0001245
 0.993782  0.000137324
 0.99248   0.00015454
 0.991055  0.000177007
 ⋮         
 0.99248   9.37743e-5
 0.993782  9.56154e-5
 0.994962  9.70128e-5
 0.996018  9.80432e-5
 0.99695   9.87771e-5
 0.997759  9.92784e-5
 0.998443  9.96036e-5
 0.999003  9.98015e-5
 0.999439  9.99126e-5
 0.999751  9.99689e-5
 0.999938  9.99934e-5
 1.0       0.0001

### Setup

In [3]:
uniform = Uniform2D(1.0, 5.0)
panels = doublet_panels(airfoil);

In [4]:
@time cl, kcl, cps = aero_coefficients(panels, uniform)
println("Lift Coefficient: $cl, $kcl")

  2.322285 seconds (7.62 M allocations: 390.577 MiB, 10.36% gc time)
Lift Coefficient: 0.8058097195775865, 0.8089037815211158


### Plotting

Grid parameters

In [5]:
x_domain, y_domain = (-1, 2), (-1, 1)
grid_size = 50
x_dom, y_dom = linspace(x_domain..., grid_size), linspace(y_domain..., grid_size)
grid = x_dom × y_dom

vels, pots = grid_data(panels, grid)

cp = pressure_coefficient.(vels, uniform.mag);

In [6]:
plot(airfoil[:,1], airfoil[:,2], markershape = :circle, aspect_ratio = :equal)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
upper_panels, lower_panels = split_panels(panels)
plot([ (first ∘ collocation_point)(panel) for panel in upper_panels ], [ panel.cp for panel in upper_panels ], 
     label = "Upper", markershape = :circle)
plot!([ (first ∘ collocation_point)(panel) for panel in lower_panels ], [ panel.cp for panel in lower_panels ], 
      label = "Lower", markershape = :circle, yaxis = (:flip))

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
p1 = contour(x_dom, y_dom, cp, fill = true)
plot(p1)
plot!(first.([ panel.start for panel in panels ]), last.([ panel.start for panel in panels ]), color = "black", aspect_ratio = :equal)

<!DOCTYPE html>
 
 
 Plots.jl